# mjo_compositing.ipynb
## Spencer Ressel
## 2023.01.17

***

This script is a compilation of functions that serve to recreate the figures of 
two papers, Wheeler and Hendon (2004) and Jiang et al. (2020).
The Wheeler and Hendon (2004) paper describes the creation of a real-time multivariate
MJO index (RMM), and splits the MJO into 8 phases based on the RMM index. 
The Jiang et al. (2020) paper is a broad overview of the MJO,
where the specific figure (Fig. 1) recreated is a composite of MJO-associated rainfall
anomalies during boreal winter for each MJO phase

***

### Inputs:       
#### Global 2.5° x 2.5° resolution, daily timeseries in netCDF format:
* precipitation from TRMM
* outgoing longwave radiation (OLR) data from Liebmann and Smith (1996)
* 850 hPa zonal wind from ERA5 reanalysis
* 200 hPa zonal wind from ERA5 reanalysis
                  
### Dependencies: 
* mjo_mean_state_diagnostics.py

## Imports

In [36]:
# Data processing tools
import numpy as np
import scipy
import scipy.signal as signal
import xarray as xr
import mjo_mean_state_diagnostics as mjo

# Plotting
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.gridspec import GridSpec

# Seaborn
import seaborn as sns

# # Cartopy
# import cartopy.crs as ccrs
# import cartopy.util as cutil
# from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

## Constants

In [37]:
# Set time bounds
TIME_MIN = 19990101
TIME_MAX = 20181231
SAMPLING_FREQUENCY = 1

# Set latitude bounds
LAT_MIN = -25
LAT_MAX = 25

# Set central longitude
CENTRAL_LONGITUDE = 160

# Set longitude bounds
LON_MIN = -180
LON_MAX = 180

# Cut-off periods for intraseasonal filtering
INTRASEASONAL_LOWCUT = 100
INTRASEASONAL_HIGHCUT = 20

# Seconds per day
SECONDS_PER_DAY = 24 * 3600

## Load Data

### Precipitation

In [38]:
data_directory_precip = r"/home/disk/eos7/sressel/research/data/trmm/"
file_name_precip = r"data_trmm_daily_2018.nc"
data_precipitation = xr.open_dataset(
    data_directory_precip + file_name_precip, engine="netcdf4"
)
data_precipitation = data_precipitation.rename(
    {"lat": "latitude", "lon": "longitude", "prec": "precipitation"}
)

# Assign precipitation data to variables
precipitation = data_precipitation["precipitation"]
time = data_precipitation["time"]
latitude = data_precipitation["latitude"]
longitude = data_precipitation["longitude"]


### OLR Data

In [39]:
data_directory_olr = r"/home/disk/eos7/sressel/research/data/noaa_olr/"
file_name_olr = r"olr.day.mean.nc"
data_olr = xr.open_dataset(data_directory_olr + file_name_olr, engine="netcdf4")
data_olr = data_olr.rename({"lat": "latitude", "lon": "longitude"})

# Assign OLR data to variables
olr = data_olr["olr"].isel(latitude=slice(None, None, -1))

# Convert OLR time units from datetime64 to YYYYMMDD
olr["time"] = mjo.datetime64_to_yyyymmdd(olr["time"].values)
olr["time"].attrs["units"] = "YYYYMMDD"
olr["time"].attrs["long_name"] = "time"

# Assign OLR data to variables
olr = data_olr["olr"].isel(latitude=slice(None, None, -1))

# Convert OLR time units from datetime64 to YYYYMMDD
olr["time"] = mjo.datetime64_to_yyyymmdd(olr["time"].values)
olr["time"].attrs["units"] = "YYYYMMDD"
olr["time"].attrs["long_name"] = "time"


### Zonal Wind

In [40]:
data_directory_zonal_wind = r"/home/disk/eos7/sressel/research/data/era5/"
file_name_zonal_wind = r"daily_2_5_degree_zonal_wind_pressure_levels_1979_2020.nc"
data_zonal_wind = xr.open_dataset(
    data_directory_zonal_wind + file_name_zonal_wind, engine="netcdf4"
)

zonal_wind = data_zonal_wind["u"].isel(latitude=slice(None, None, -1))
zonal_wind["time"] = mjo.datetime64_to_yyyymmdd(zonal_wind["time"].values)
zonal_wind_longitudes = zonal_wind["longitude"].values
zonal_wind_longitudes[zonal_wind_longitudes < 0] = (
    zonal_wind_longitudes[zonal_wind_longitudes < 0] + 360
)
zonal_wind["longitude"] = zonal_wind_longitudes
zonal_wind = zonal_wind.sortby(zonal_wind.longitude)

upper_level_zonal_wind = zonal_wind.sel(level=200)
lower_level_zonal_wind = zonal_wind.sel(level=850)


### Meridional Wind

In [41]:
data_directory_meridional_wind = r"/home/disk/eos7/sressel/research/data/era5/"
file_name_meridional_wind = (
    r"daily_2_5_degree_meridional_wind_pressure_levels_1979_2020.nc"
)
data_meridional_wind = xr.open_dataset(
    data_directory_meridional_wind + file_name_meridional_wind, engine="netcdf4"
)

meridional_wind = data_meridional_wind["v"].isel(latitude=slice(None, None, -1))
meridional_wind["time"] = mjo.datetime64_to_yyyymmdd(meridional_wind["time"].values)
meridional_wind_longitudes = meridional_wind["longitude"].values
meridional_wind_longitudes[meridional_wind_longitudes < 0] = (
    meridional_wind_longitudes[meridional_wind_longitudes < 0] + 360
)
meridional_wind["longitude"] = meridional_wind_longitudes
meridional_wind = meridional_wind.sortby(meridional_wind.longitude)

upper_level_meridional_wind = meridional_wind.sel(level=200)
lower_level_meridional_wind = meridional_wind.sel(level=850)

***
***

## Subset Data
Specifically select the data from times of interest and from tropical latitudes

### Precipitation

In [42]:
tropical_precipitation = precipitation.sel(
    time=slice(TIME_MIN, TIME_MAX), latitude=slice(LAT_MIN, LAT_MAX)
)

### OLR

In [43]:
tropical_olr = olr.sel(
    time=slice(TIME_MIN, TIME_MAX), latitude=slice(LAT_MIN, LAT_MAX),
)

### Zonal Wind

#### Lower Level - 850 hPa

In [44]:
tropical_lower_level_zonal_wind = lower_level_zonal_wind.sel(
    time=slice(TIME_MIN, TIME_MAX), latitude=slice(LAT_MIN, LAT_MAX),
)

#### Upper Level - 200 hPa

In [45]:
tropical_upper_level_zonal_wind = upper_level_zonal_wind.sel(
    time=slice(TIME_MIN, TIME_MAX), latitude=slice(LAT_MIN, LAT_MAX),
)

### Meridional Wind

#### Lower Level

In [46]:
tropical_lower_level_meridional_wind = lower_level_meridional_wind.sel(
    time=slice(TIME_MIN, TIME_MAX), latitude=slice(LAT_MIN, LAT_MAX),
)

#### Upper Level

In [47]:
tropical_upper_level_meridional_wind = upper_level_meridional_wind.sel(
    time=slice(TIME_MIN, TIME_MAX), latitude=slice(LAT_MIN, LAT_MAX),
)

### Time and Latitude

In [92]:
# Subset data using a for loop
variables_dict = {"precipitation":precipitation,
                  "olr":olr, 
                  "lower_level_zonal_wind":lower_level_zonal_wind, 
                  "upper_level_zonal_wind":upper_level_zonal_wind, 
                  "lower_level_meridional_wind":lower_level_meridional_wind,
                  "upper_level_meridional_wind":upper_level_meridional_wind}

tropically_subset_variables = {}
tropical_variable_anomalies = {}
tropical_variable_anomalies_detrended = {}
intraseasonal_filtered_tropical_variable = {}
meridional_mean_intraseasonal_variable = {}

for variable in variables_dict:
    tropically_subset_variables[variable] = variables_dict[variable].sel(time=slice(TIME_MIN, TIME_MAX), latitude=slice(LAT_MIN, LAT_MAX)
)
    [tropical_variable_anomalies[variable], _] = mjo.remove_annual_cycle_matrix(tropically_subset_variables[variable], time=time, lat=latitude_tropics, lon=longitude)
    
    tropical_variable_anomalies_detrended[variable] = signal.detrend(tropical_variable_anomalies[variable], axis=0)
    
    intraseasonal_filtered_tropical_variable[variable] = mjo.lanczos_bandpass_filter(
    tropical_variable_anomalies_detrended[variable],
    lowcut=(1 / INTRASEASONAL_LOWCUT),
    highcut=(1 / INTRASEASONAL_HIGHCUT),
    fs=SAMPLING_FREQUENCY,
    filter_axis=0,
)
    
    meridional_mean_intraseasonal_variable[variable] = np.mean(
    intraseasonal_filtered_tropical_variable[variable], axis=1
)   

In [48]:
# Time
time = time.sel(time=slice(TIME_MIN, TIME_MAX))
[year, month, day] = mjo.yyyymmdd_y_m_d(time.values)

# Latitude
latitude_tropics = latitude.sel(latitude=slice(LAT_MIN, LAT_MAX))

***
***

## Process Data

Remove the annual cycle and the first three harmonics (seasonal cycle)

### Precipitation

In [49]:
[
    tropical_precipitation_anomalies,
    tropical_precipitation_cyc,
] = mjo.remove_annual_cycle_matrix(
    tropical_precipitation, time=time, lat=latitude_tropics, lon=longitude
)

### OLR

In [56]:
[
    tropical_olr_anomalies, tropical_olr_cyc
] = mjo.remove_annual_cycle_matrix(
    tropical_olr, time=time, lat=latitude_tropics, lon=longitude
)

### Zonal Wind

#### Lower Level

In [52]:
[
    tropical_lower_level_zonal_wind_anomalies,
    tropical_olr_cyc,
] = mjo.remove_annual_cycle_matrix(
    tropical_lower_level_zonal_wind, time=time, lat=latitude_tropics, lon=longitude
)

#### Upper Level

In [53]:
[
    tropical_upper_level_zonal_wind_anomalies,
    tropical_olr_cyc,
] = mjo.remove_annual_cycle_matrix(
    tropical_upper_level_zonal_wind, time=time, lat=latitude_tropics, lon=longitude
)

### Meridional Wind

#### Lower Level

In [54]:
[
    tropical_lower_level_meridional_wind_anomalies,
    tropical_olr_cyc,
] = mjo.remove_annual_cycle_matrix(
    tropical_lower_level_meridional_wind, time=time, lat=latitude_tropics, lon=longitude
)

#### Upper Level

In [55]:
[
    tropical_upper_level_meridional_wind_anomalies,
    tropical_olr_cyc,
] = mjo.remove_annual_cycle_matrix(
    tropical_upper_level_meridional_wind, time=time, lat=latitude_tropics, lon=longitude
)

## Detrend Data

Specifically remove any linear trends in time

### Precipitation

In [61]:
tropical_precipitation_anomalies_detrended = signal.detrend(
    tropical_precipitation_anomalies, axis=0
)

### OLR

In [62]:
tropical_olr_anomalies_detrended = signal.detrend(tropical_olr_anomalies, axis=0)


### Zonal Wind

#### Lower Level

In [63]:
tropical_lower_level_zonal_wind_anomalies_detrended = signal.detrend(
    tropical_lower_level_zonal_wind_anomalies, axis=0
)

#### Upper Level

In [64]:
tropical_upper_level_zonal_wind_anomalies_detrended = signal.detrend(
    tropical_upper_level_zonal_wind_anomalies, axis=0
)

### Meridional Wind

#### Lower Level

In [65]:
tropical_lower_level_meridional_wind_anomalies_detrended = signal.detrend(
    tropical_lower_level_meridional_wind_anomalies, axis=0
)

#### Upper Level

In [66]:
tropical_upper_level_meridional_wind_anomalies_detrended = signal.detrend(
    tropical_upper_level_meridional_wind_anomalies, axis=0
)

## Filter Data 

Temporally filter the data on intraseasonal (20-100 day) timescales, using a Lanczos filter